# MIDS W261 Spring 2016 Homework Week 3

**Ted Dunmire**  glenn.dunmire.iv@gmail.com<br/>
**Filip Krunic**  fkrunic@ischool.berkeley.edu<br />
**Ron Cordell** ron.cordell@ischool.berkeley.edu<br />

W261-4<br />
January 26, 2016

## HW3.0.

**What is a merge sort? Where is it used in Hadoop?**

A merge sort merges sorted lists into a single sorted list. The merge sort works by establishing a pointer to the beginning of each sorted list as well as a new "merge" list. The objects or keys in each list referenced by the pointers are compared and the chosen one moved or copied to the location indicated by the pointer of the merge list. The merge list pointer is advanced as is the pointer for the list from which the object was moved. This is repeated until all objects in all list have been moved or copied to the new merge list. The comparator function "chooses" the object from the source lists based on the rules coded into the comparator function such as the largest, the smallest, etc.

Hadoop uses a merge sort during the shuffle process when it takes output from multiple sources such as mappers or combiners and merges them into the sorted streams used by downstream processes.

**How is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.**

A combiner function is a function that can be used by Hadoop anywhere between the mappers and producers to help eliminate network and data traffic, especially as part of the shuffle. A an example combiner function typically provides a partial aggregation point for data emitted from the mapper to reduce hotspots in the shuffle.

An example where a combiner can be used to good effect is in a word count scenario, where the mapper emits the word as key and a value of 1. A combiner can perform aggregations on the key-value pairs by combining those with the same key and adding their values. This greatly reduces the granularity of the data required to shuffle and sort and provide to the reducers and helps reduce the amount of network and disk traffic of the shuffle, decreasing the overall run time.

**What is the Hadoop shuffle?**

![MapReduce Workflow](https://rawgit.com/dunmireg/HW3Submission/master/MapReduceWorkflow.png)

The Hadoop shuffle takes the mapper outpus, merges them and sorts them, computes a hash to partition them, and routes each partition to a reducer. It corresponds the the shuffle and sort section of the above diagram, between the mappers and reducers.

-----

## HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)**

**The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:**

    Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

**Here’s is the first few lines of the  of the Consumer Complaints  Dataset:**

    Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
    1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
    1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
    1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
    1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

**Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).**

### HW3.1 Map Function

In [1]:
%%writefile mapper.py
#!/usr/bin/python
import sys

line_num = 0
for line in sys.stdin: #read input
    if line_num == 0:
        line_num += 1 #skip the first line, which is a header
        continue
    else:
        line = line.strip() #remove extra chars
        line = line.rstrip()
        line = line.split(',') #split on comma delimiter
        if line[1] == "Debt collection": #line[1] is the issue part of the complaint
            sys.stderr.write('reporter:counter:Debt-Counter,Total,1\n') #increment counter based on complaint
        elif line[1] == 'Mortgage':
            sys.stderr.write('reporter:counter:Mortgage-Counter,Total,1\n')
        else:
            sys.stderr.write('reporter:counter:Other-Counter,Total,1\n') #all other issues are lumped together
        sys.stderr.write("reporter:counter:Tokens,Total,1\n")
        print line[1] + '\t' + '1' #This just prints the issue and a 1. Will use to check if counters are correct

Overwriting mapper.py


### HW3.1 Reduce Function

In [2]:
%%writefile reducer.py
#!/usr/bin/python
import sys

#keep counters to see how many results we have
debt_counter = 0
mortgage_counter = 0
other_counter = 0

for line in sys.stdin: #read input
    line = line.strip().split('\t')
    if line[0] == "Debt collection": #recall we passed the issue as line[0] from the mapper, here we parse 
        debt_counter +=1  
    elif line[0] == 'Mortgage':
        mortgage_counter += 1
    else:
        other_counter += 1
print "Debt collection: " + str(debt_counter) #print results. These should match the counters from the mapper
print "Mortgage: " + str(mortgage_counter)
print "Other: " + str(other_counter)

Overwriting reducer.py


### HW 3.1 Start Hadoop

In [3]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/01/29 21:04:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/01/29 21:05:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [4]:
#add input to hdfs
!hdfs dfs -put Consumer_Complaints.csv /user/dunmireg

16/01/29 21:05:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### HW3.1 Hadoop MapReduce

In [5]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper.py \
-reducer reducer.py \
-input Consumer_Complaints.csv \
-output consumer_counters

16/01/29 21:05:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/29 21:05:34 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/29 21:05:34 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/29 21:05:34 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/29 21:05:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/29 21:05:35 INFO mapreduce.JobSubmitter: number of splits:1
16/01/29 21:05:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1206054309_0001
16/01/29 21:05:35 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/29 21:05:35 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/29 21:05:35 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.Fi

In [6]:
#show results
!hdfs dfs -cat /user/dunmireg/consumer_counters/part-00000

16/01/29 21:06:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Debt collection: 44372	
Mortgage: 125752	
Other: 142789	


**Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. **

![Job Tracker](https://rawgit.com/dunmireg/HW3Submission/master/jobtrackercounters.png)

Notice that the mapper counters and the results from counting in the reducer match. In total we found: 

__Debt collection: 44372	
Mortgage: 125752	
Other: 142789__	

-----

## HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

### HW3.2 Part 1

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.


** Write input string to a file on disk **

In [1]:
#make a basic text file to test
with open('input_text.txt', 'w') as myfile:
    myfile.write('foo foo quux labs foo bar quux')

### HW3.2 Word Count Map Function

In [8]:
%%writefile mapper.py
#!/usr/bin/python
import sys

sys.stderr.write('reporter:counter:Map-Count,Total,1\n') #increment mapper counter
for line in sys.stdin: #read input
    line = line.strip()
    line = line.rstrip()
    line = line.split() #split on space delimiter
    for word in line:
        print word + '\t' + '1' #emit word and count of 1

Overwriting mapper.py


### HW3.2 Word Count Reducer

In [9]:
%%writefile reducer.py
#!/usr/bin/python
import sys

current_word = None #keep track of current word  and current count
current_count = None
word = None
sys.stderr.write('reporter:counter:Reduce-Counter,Total,1\n') #increment reducer counter
for line in sys.stdin:
    line = line.strip()
    line = line.rstrip()
    word, count = line.split('\t') #split here on tab
    
    if current_word == word: #increment the counter
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count) #print results when we have found a new word
        current_word = word #change word
        current_count = int(count)

#print final word
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [19]:
#!cat input.txt | python mapper.py | sort | python reducer.py

### HW3.2 Hadoop MapReduce

In [1]:
# create input file and add input to hdfs
!echo "foo foo quux labs foo bar quux" >input_text.txt
!hdfs dfs -put input_text.txt /user/dunmireg

16/02/01 21:00:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
#run hadoop, manually setting the number of mappers and reducers
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.map.tasks=1 \
-D mapred.reduce.tasks=4 \
-mapper mapper.py \
-reducer reducer.py \
-input input_text.txt \
-output word_count

16/02/01 22:33:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:33:49 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 22:33:49 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 22:33:49 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 22:33:49 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 22:33:49 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 22:33:49 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/02/01 22:33:49 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 22:33:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local927319060_0001
16/02/01 22:33:49 INFO mapr

** RESULT: Mappers - 1, Reducers - 4 **

### HW3.2 Part 2

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

### HW3.2 Part 2 Map Function

In [182]:
%%writefile mapper32b.py
#!/usr/bin/env python
#
# W261 HW 3.2 MapReduce and Counters for Code Analysis
# Read from a CSV file of consumer complaints with fields as follows:
#
# Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,
# Date sent to company,Company,Company response,Timely response?,Consumer disputed?
#
# Use counters to count the number of times the mapper is called
#
# Remember that in Hadoop streaming, to update a counter is to write to STDERR in the format
# reporter:counter:<group>,<counter>,<amount>

import sys
import re

WORD_RE = re.compile(r"[\w']+")

# Read data from STDIN and use counters to count the data
def main(separator='\t'):   
    for line in sys.stdin:
        fields = line.split(',')
        try:
            # check to see if this is a header by trying to convert the first field to an integer
            id = int(fields[0])
            # we have a real record, so do some mapping
            for word in WORD_RE.findall(fields[3]):
                sys.stdout.write('{0}{1}{2}\n'.format(word, separator, 1))
        except:
            # must be a header record so skip it
            pass

if __name__ == "__main__":
    # increment counter for mapper call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,mapper,1\n")
    main()

Overwriting mapper32b.py


### HW3.2 Part Reduce Function

In [194]:
%%writefile reducer32b.py
#!/usr/bin/env python

from itertools import groupby
from operator import itemgetter
import sys

def read(file, separator='\t'):
    for line in file:
        yield line.rstrip().split(separator, 1)

def main(separator='\t'):
    # input comes from STDIN (standard input)
    data = read(sys.stdin, separator=separator)
    # groupby groups multiple word-count pairs by word,
    # and creates an iterator that returns consecutive keys and their group:
    #   current_word - string containing a word (the key)
    #   group - iterator yielding all ["&lt;current_word&gt;", "&lt;count&gt;"] items
    for current_word, group in groupby(data, itemgetter(0)):
        try:
            total_count = sum(int(count) for current_word, count in group)
            sys.stderr.write("reporter:counter:Code Call Counters,reducer pairs,1\n")
            sys.stdout.write("{0}{1}{2}\n".format(current_word, separator, total_count))
        except ValueError:
            sys.stderr.write("reporter:counter:Code Call Counters,reducer skipped pairs,1\n")
            # count was not a number, so silently discard this item
            pass

if __name__ == "__main__":
    # increment counter for reducer call, write to STDERR
    sys.stderr.write("reporter:counter:Code Call Counters,reducer,1\n")
    main()

Overwriting reducer32b.py


### HW3.2 Part 2 Hadoop MapReduce

In [184]:
!chmod a+x mapper32b.py
!chmod a+x reducer32b.py

In [196]:
!hdfs dfs -rm -r /user/rcordell/recordsOutput
!yarn jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
    -D mapreduce.job.reduces=2 \
    -D mapreduce.job.maps=2 \
    -files "mapper32b.py,reducer32b.py" \
    -mapper mapper32b.py \
    -reducer reducer32b.py \
    -input Consumer_Complaints.csv \
    -output recordsOutput

16/01/30 19:16:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/hadoop-unjar3393828723429341869/] [] /var/folders/z_/rfp5q2cd6db13d19v6yw0n8w0000gn/T/streamjob6639340843874969134.jar tmpDir=null
16/01/30 19:16:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 19:16:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/01/30 19:16:44 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/30 19:16:44 INFO mapreduce.JobSubmitter: number of splits:2
16/01/30 19:16:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1454175435207_0018
16/01/30 19:16:45 INFO impl.YarnClientImpl: Submitted application application_1454175435207_0018
16/01/30 19:16:45 INFO mapreduce.Job: The url to track the job: http://Rons-iMac-Retina.local:8088/proxy/application_1454175435207_0018/
16/01/3

In [197]:
!hdfs dfs -cat /user/rcordell/recordsOutput/part-00000 | wc -l

16/01/30 19:17:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
      86


** RESULTS: Mapper - 2, Reducer - 2, Unique Words - 86 **

### HW3.2 Part 3

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

### HW3.2 Part 3 - Combiner Function

The combiner is essentially the same as the aggregation part of the reducer.

In [13]:
%%writefile combiner.py
#!/usr/bin/python
import sys

#we added a combiner here to perform the exact same function as the reducer above, the only difference being the counter
current_word = None
current_count = None
word = None
sys.stderr.write('reporter:counter:Combiner-Counter,Total,1\n') #increment combiner counter
for line in sys.stdin:
    line = line.split('\t') #split line on tab from standard input
    word = line[0]
    count = int(line[1])
    
    if current_word == word: #increment word count
        current_count += count
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count) #print result when found a new word
        current_word = word
        current_count = count

#print last word
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Writing combiner.py


### HW3.2 Part 3 Hadoop MapReduce

In [18]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.reduce.tasks=2 \
-D mapred.map.tasks=2 \
-mapper mapper.py \
-reducer reducer.py \
-combiner combiner.py \
-input Consumer_Complaints.csv \
-output word_count

16/02/01 22:36:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:36:51 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 22:36:51 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 22:36:51 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 22:36:51 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 22:36:51 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 22:36:52 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/02/01 22:36:52 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 22:36:52 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1588555268_0001
16/02/01 22:36:52 INFO map

** RESULTS: Mapper - 1, Combiner - 2, Reducer - 2 **

### HW3.2 Part 4

Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

### HW3.2 Part 4 Stage 1 Map Function

In [2]:
%%writefile mapper-3-2-4.py
#!/usr/bin/python
import sys
import re
from csv import reader

#Structure of complaints
#Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,
#Company,Company response,Timely response?,Consumer disputed?

line_num = 0 #for skipping header
total = 0 #total number of words in issue
WORD_RE = re.compile(r"[\w']+")
for line in reader(sys.stdin): #here we use csv.reader to read the input of the file 
    if line_num == 0: #skip first row, which is a header
        line_num += 1
        continue
    else:
        issue = line[3] #parse the issue of the complaint
        if issue == '': #There are exactly four records where the issue was marked as blank. 
            issue = 'Blank' #We felt that setting to blank was appropriate
        words = re.findall(WORD_RE, issue)
        for word in words:
            total += 1 #increment total word counter
            print word.lower() + '\t' + str(1) #print the word and a count of 1
print '*' + '\t' + str(total) #use order inversion to provide total as first input to reducer

Overwriting mapper-3-2-4.py


### HW3.2 Part 4 Stage 1 Reduce Function

In [7]:
%%writefile reducer-3-2-4.py
#!/usr/bin/python
import sys
import operator

current_word = None #follows same basic structure as word count we have worked with before
current_count = None
word = None
total = 0
#wordcount = {} #a dictionary to store counts. This was used in an earlier version using an in-memory mapper

for line in sys.stdin:
    line = line.split('\t') #split line
    word = line[0] #read word
    count = int(line[1]) #get count
    if word == '*':
        total = count #if the word is * we know this is the total number of words and set this as a field
    else: #otherwise continue as normal
        if current_word == word: 
            current_count += count
        else:
            if current_word:
                #wordcount[current_word] = current_count #used in in-memory dictionary version
                #structure of result is word + count + relative count
                print current_word + '\t' + str(current_count) + '\t' + str(float(current_count)/total) #print result
            current_word = word
            current_count = count

#print last word
if current_word == word:
    #wordcount[current_word] = current_count
    print current_word + '\t' + str(current_count) + '\t' + str(float(current_count)/total)
    
#Code for in-memory dictionary printing
# largest = 50
# smallest = 10
# sortedWordCount = sorted(wordcount.items(), key = operator.itemgetter(1))

# print "The Top 50 terms are"
# for i in range(largest):
#     print str(sortedWordCount[-i-1][0]) + '\t' + str(sortedWordCount[-i-1][1]) + '\t' + str(float(sortedWordCount[-i-1][1])/total)
    
# print '\n'

# print "The bottom 10 terms are"
# for i in range(smallest):
#     print str(sortedWordCount[i][0]) + '\t' + str(sortedWordCount[i][1]) + '\t' + str(float(sortedWordCount[i][1])/total)

Overwriting reducer-3-2-4.py


### HW3.2 Part 4 Stage 2 Map Function

In [3]:
%%writefile mapper-3-2-4-2.py
#!/usr/bin/python
import sys

#Structure is word + \t + count + \t + relative count, to be read from output of first set of jobs

for line in sys.stdin:
    line = line.strip()
    line = line.split('\t') #split the line
    print line[1] + '\t'+ line[0] + '\t' + line[2] #now we are using the number as the key, this will be sorted


Overwriting mapper-3-2-4-2.py


### HW3.2 Part 4 Stage 2 Reduce Function

In [10]:
%%writefile reducer-3-2-4-2.py
#!/usr/bin/python
import sys

print "Issue" + '\t' + "Count" + '\t' + 'Relative Count'
for line in sys.stdin: #reads result of second mapper using number as key
    line = line.strip()
    line = line.split('\t')
    #structure currently is count + word + relative count
    print line[1] + '\t' + line[0] + '\t'+ line[2] #now reverse back to display word + count + relative count

Overwriting reducer-3-2-4-2.py


In [11]:
#!cat Consumer_Complaints.csv | python mapper-3-2-4.py | sort | python reducer-3-2-4.py | python mapper-3-2-4-2.py | sort -n | python reducer-3-2-4-2.py> output.txt

### HW3.2 Part 4 Hadoop MapReduce Stage 1 - Word Counts and Frequencies

In [14]:
#run hadoop job 
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper-3-2-4.py \
-reducer reducer-3-2-4.py \
-input Consumer_Complaints.csv \
-output word_count

16/01/31 23:01:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:01:12 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 23:01:12 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 23:01:12 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 23:01:13 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 23:01:13 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 23:01:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1964160718_0001
16/01/31 23:01:13 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/31 23:01:13 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/31 23:01:13 INFO mapreduce.Job: Running job: job_local1964160718_0001
16/01/31 23:01:1

### HW3.2 Part 4 Hadoop MapReduce Stage 2 - Final Sort

In [15]:
#run second job to properly sort by counts
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.num.map.output.key.fields=2 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D  mapred.text.key.comparator.options="-k1nr -k2" \
-mapper mapper-3-2-4-2.py \
-reducer reducer-3-2-4-2.py \
-input /user/dunmireg/word_count/part-00000 \
-output sortedWordCount

16/01/31 23:01:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:01:45 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 23:01:45 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 23:01:45 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 23:01:45 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 23:01:45 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 23:01:46 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/01/31 23:01:46 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/31 23:01:46 INFO mapreduce.JobSubmitter: Su

### HW3.2 Part 4 Results

In [ ]:
#show results, display top 50 words
!hdfs dfs -cat /user/dunmireg/sortedWordCount/part-00000 | head -50

16/02/02 23:03:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable

loan	119630	0.0887257548698

collection	72394	0.0536923204718

foreclosure	70487	0.0522779594041

modification	70487	0.0522779594041

account	57448	0.0426073490409

credit	55251	0.0409779042239

or	40508	0.0300434914174

payments	39993	0.0296615323456

escrow	36767	0.0272689110532

servicing	36767	0.0272689110532

report	34903	0.0258864417138

incorrect	29133	0.0216070167736

information	29069	0.0215595500151

on	29069	0.0215595500151

debt	27874	0.020673256635

closing	19000	0.0140916939106

not	18477	0.013703801494

attempts	17972	0.0133292591032

collect	17972	0.0133292591032

cont'd	17972	0.0133292591032

owed	17972	0.0133292591032

and	16448	0.012198956918

management	16205	0.0120187315695

opening	16205	0.0120187315695

of	13983	0.0103707450501

my	10731	0.00795884038709

deposits	10555	0.00782830680139

withdrawals	10555	0.00782830680139

problems	9484	0.0070339802657

application	8868	0.00657711271575

communication	8671	0.00643100409994

tactics	8671	0.00643100409994

broker	8625	0.00639688736732

mortgage	8625	0.00639688736732

originator	8625	0.00639688736732

to	8401	0.00623075371279

unable	8178	0.00606536172637

billing	8158	0.00605052836435

other	7886	0.005848794641

disclosure	7655	0.00567746930977

verification	7655	0.00567746930977

disputes	6938	0.00514569328167

reporting	6559	0.00486460107156

lease	6337	0.00469995075324

the	6248	0.00463394229229

being	5663	0.00420006645346

by	5663	0.00420006645346

caused	5663	0.00420006645346

funds	5663	0.00420006645346

low	5663	0.00420006645346

In [ ]:
#Display bottom 10 counts (least is on top)
!hdfs dfs -cat /user/dunmireg/sortedWordCount/part-00000 | tail | sort -k2

16/02/02 23:06:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable

blank	4	2.96667240223e-06

disclosures	64	4.74667584357e-05

missing	64	4.74667584357e-05

amt	71	5.26584351396e-05

day	71	5.26584351396e-05

checks	75	5.56251075419e-05

convenience	75	5.56251075419e-05

credited	92	6.82334652514e-05

payment	92	6.82334652514e-05

amount	98	7.26834738547e-05

## HW3.3. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 

    FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
    GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
    ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
    ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
    ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

### HW3.3 Mapper

Below is the mapper used to accomplish this task. It emits each token along with a basket size to the reducers for additional processing. 

In [73]:
%%writefile mapperQ33.py
#!/usr/bin/env python

import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":


	# Read input 
	data = readInput(sys.stdin)
	for line in data: 
		holdingDict = defaultdict(int)
		basketSize = len(line)


		# Append elements 
		for token in line: 
			holdingDict[token] += 1


		# Emit results 
		for k, v in holdingDict.iteritems(): 
			basketInfo = str([v, basketSize])
			print '%s%s%s' % (k, '\t', basketInfo)
            
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

Overwriting mapperQ33.py


### HW3.3 Reducer

Below is the reducer used to accomplish this task. The reducer uses a `defaultdict` object from `collections` to automatically collate tokens and their counts without having to explicitly instantiate the key. This is convenient when reading from `sys.stdin` as one can yield lines and store the tokens simultaneously. 

In [74]:
%%writefile reducerQ33.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 

def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)
	maxBasket = 0
	totalTerms = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 


		# Parse value 
		token = line[0]
		termCount, basketSize = ast.literal_eval(line[1])
		totalTerms += termCount


		# Store results 
		storingDict[token] += termCount
		maxBasket = max(maxBasket, basketSize)


	# Metrics
	numUniqueProducts = len(set(storingDict.keys()))
	largestBasket = maxBasket


	# Compute frequencies 
	for k, v in storingDict.iteritems():
		storingDict[k] = v

	
	# Find most frequent terms 
	mostFrequentTerms = [(k, v, round(v / totalTerms, 4)) for k, v in storingDict.iteritems()]
	mostFrequentTerms = sorted(mostFrequentTerms, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Number of Unique Products ==========' + '\n'
	print 'Answer: ' + str(numUniqueProducts) + '\n'
    
	print '\n' + '========= Largest Basket ==========' + '\n'
	print 'Answer: ' + str(largestBasket) + '\n'    

	print '========== Most Frequent Terms ==========' + '\n'
	template = "{0:20}|{1:20}|{2:20}"
	print template.format("ITEM", "FREQUENCY", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mostFrequentTerms[:50]: 
		print template.format(*termPair)
        
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Reducers,1\n")

Overwriting reducerQ33.py


### HW3.3 Wrapper

Below is the bash script that's used to submit the Hadoop Streaming job. It maps user input to variables and specifies the options for the job. 

In [29]:
%%writefile wrapperQ33.sh
#!/bin/bash


# Initialize
RAW_DATA=$1
RAW_MAPPER=$2
RAW_REDUCER=$3


# Hadoop variables 
HDFS_DIR="/user/john/notebook"
HDFS_INPUT="$HDFS_DIR/input"
HDFS_OUTPUT="$HDFS_DIR/output"
HDFS_FILES="$HDFS_DIR/files"


# Local variables 
PROJECT_DIR="/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook"
DATA="$PROJECT_DIR/$RAW_DATA"
MAPPER="$PROJECT_DIR/$RAW_MAPPER"
REDUCER="$PROJECT_DIR/$RAW_REDUCER"


# NAIVE="$PROJECT_DIR/naiveBayes.py"
STREAMING_JAR="$PROJECT_DIR/hadoop-streaming-2.6.0.jar"


# Make directories and put file 
hdfs dfs -rm -r $HDFS_DIR
hdfs dfs -mkdir $HDFS_DIR $HDFS_INPUT $HDFS_FILES
hdfs dfs -put $DATA $HDFS_INPUT


# Execute
hadoop jar $STREAMING_JAR \
	-file "$MAPPER" -mapper "$MAPPER" \
	-file "$REDUCER" -reducer "$REDUCER" \
	-input $HDFS_INPUT \
	-output $HDFS_OUTPUT 


# Output results  
if [ $? -eq 0 ]; then 
	hdfs dfs -cat $HDFS_OUTPUT/part-00000
fi	

Overwriting wrapperQ33.sh


### HW3.3 Hadoop MapReduce Submit Job

Below we submit the job. The wrapper takes as arguments the data, mapper, and reducer. Later versions of the wrapper also allow combiner arguments. 

In [40]:
!bash wrapperQ33.sh ProductPurchaseData.txt mapperQ33.py reducerQ33.py

16/01/30 16:05:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 16:05:42 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/01/30 16:05:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 16:05:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 16:05:49 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/30 16:05:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ33.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

-----

## HW3.4

*Write a map-reduce program to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.*

*List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. *

*Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.*

## Solution Approach A

### Mapper

Below is the mapper code for the pairs implementation. It generates tuple-combinations from the input lines, sorts them to get unique keys, then emits them to the reducer or optional combiner. 

In [10]:
%%writefile mapperQ34.py
#!/usr/bin/env python

from __future__ import division 
import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	# Read input 
	data = readInput(sys.stdin)
	totalBaskets = 0

	for line in data: 

		# Increment
		totalBaskets += 1

		# Get unique keys 
		pairs = list(combinations(line, 2))
		

		# Sort keys 
		sortedPairs = []
		for pair in pairs: 
			pList = list(pair)
			pList.sort()
			sortedPairs.append(tuple(pList))

		# Emit
		for pair in sortedPairs: 
			print '%s%s%s' % (pair, '\t', 1)
			
		
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

	# Emit basket count 
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting mapperQ34.py


### HW3.4 Reducer

Below is the reducer for this process. It uses a simple key-aggregation to collate the results from the mappers, where the keys are unique tuples from the lines read in by the mapper. The uniqueness condition is specified at the line-level when read by the mapper. 

In [11]:
%%writefile reducerQ34.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 


def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)
	support = 100
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		# Check for basket 
		token = line[0]

		if token == '*': 
			totalBaskets += int(line[1])

		else:

			# Parse 
			termCount = int(line[1])

			# Store results 
			storingDict[token] += termCount

	
	# Filter
	filterDict = defaultdict(int)
	for k, v in storingDict.iteritems(): 
		if v >= support:
			filterDict[k] += v 


	# Find most frequent terms 
	mostFrequentTerms = [(k, v, round(int(v) / totalBaskets, 3)) for k, v in filterDict.iteritems()]
	mostFrequentTerms = sorted(mostFrequentTerms, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Terms ==========' + '\n'
	template = "{0:30}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mostFrequentTerms[:50]: 
		print template.format(*termPair)
        
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Reducers,1\n")

Overwriting reducerQ34.py


### HW3.4 Combiner (Optional)

Below is the code for an optional combiner. It has the same signature as the reducer and does the same essential aggregation. 

In [12]:
%%writefile combinerQ34.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 


def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)


	# Read data 
	data = readInput(sys.stdin)
	totalBaskets = 0

	
	for line in data: 

		# Check for basket 
		token = line[0]

		if token == '*': 
			totalBaskets += int(line[1])

		else:

			# Parse 
			termCount = int(line[1])

			# Store results 
			storingDict[token] += termCount


	# Emit
	for k, v in storingDict.iteritems(): 
		print '%s%s%s' % (k, '\t', v)
        
	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Combiners,1\n")

	# Emit basket count 
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting combinerQ34.py


### HW3.4 Wrapper

Here is the wrapper for this particular submission. The noticeable difference is that it now has been modified to pass a combiner file to Hadoop Streaming. 

In [52]:
%%writefile wrapperQ34.sh
#!/bin/bash


# Initialize
RAW_DATA=$1
RAW_MAPPER=$2
RAW_REDUCER=$3
RAW_COMBINER=$4


# Hadoop variables 
HDFS_DIR="/user/john/notebook"
HDFS_INPUT="$HDFS_DIR/input"
HDFS_OUTPUT="$HDFS_DIR/output"
HDFS_FILES="$HDFS_DIR/files"


# Local variables 
PROJECT_DIR="/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook"
DATA="$PROJECT_DIR/$RAW_DATA"
MAPPER="$PROJECT_DIR/$RAW_MAPPER"
REDUCER="$PROJECT_DIR/$RAW_REDUCER"
COMBINER="$PROJECT_DIR/$RAW_COMBINER"


# NAIVE="$PROJECT_DIR/naiveBayes.py"
STREAMING_JAR="$PROJECT_DIR/hadoop-streaming-2.6.0.jar"


# Make directories and put file 
hdfs dfs -rm -r $HDFS_DIR
hdfs dfs -mkdir $HDFS_DIR $HDFS_INPUT $HDFS_FILES
hdfs dfs -put $DATA $HDFS_INPUT


# Execute
hadoop jar $STREAMING_JAR \
	-file "$MAPPER" -mapper "$MAPPER" \
	-file "$REDUCER" -reducer "$REDUCER" \
    -file "$COMBINER" -combiner "$COMBINER" \
	-input $HDFS_INPUT \
	-output $HDFS_OUTPUT \
	-cmdenv mapred.map.max.attempts=1 \
	-cmdenv mapred.reduce.max.attempts=1 \


# Output results  
if [ $? -eq 0 ]; then 
	hdfs dfs -cat $HDFS_OUTPUT/part-00000
fi	

Overwriting wrapperQ34.sh


##### Submit Job

We now submit the job for this process. Notice that an argument has been added for the combiner. 

In [13]:
!bash wrapperQ34.sh ProductPurchaseData.txt mapperQ34.py reducerQ34.py combinerQ34.py

16/02/01 19:56:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:56:53 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/02/01 19:56:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:56:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:57:01 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/01 19:57:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ34.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

To view the counters, check the 'User-Defined' category in the job logs appearing just prior to the reducer results. It should appear as follows: 

	User-Defined
		Number of Combiners=2
		Number of Mappers=1
		Number of Reducers=1

The job in total took 1 minute and 16 seconds to complete

## HW3.4 Solution Approach B

### Mapper

In [22]:
%%writefile mapper-3-4.py
#!/usr/bin/python
import sys
from itertools import combinations

totalBaskets = 0 #field to hold total number of baskets
sys.stderr.write('reporter:counter:Mapper-Counter,Total,1\n') #increment mapper counter
for line in sys.stdin:
    totalBaskets += 1 #increment
    line = line.strip()
    line = line.split()
    
    pairs = list(combinations(line, 2)) #this give all pair combinations for all items in a basket
    for pair in pairs:
        pair = sorted(list(pair)) #sort the pairs in lexicographic order
        print pair[0] + ' ' + pair[1] + '\t' + str(1) #print result: item1 + item2 + count of 1
print '*' + '\t' + str(totalBaskets) #print total baskets

Overwriting mapper-3-4.py


### Reducer

In [23]:
%%writefile reducer-3-4.py
#!/usr/bin/python
import sys
from collections import defaultdict
import operator

support = 100 #level of support
totalBaskets = 0 #total basket
pairs = defaultdict(int) #in-memory dictionary to hold reults. This wouldn't work as a scalable solution

sys.stderr.write('reporter:counter:Reducer-Counter,Total,1\n') #increment counter
for line in sys.stdin:
    line = line.split('\t')
    if line[0] == '*': #order inversion says this will be the total 
        totalBaskets = int(line[1])
    else:
        pairs[line[0]] += int(line[1]) #increment the default dictionary for pair by counter. 
        #note when using a regular dictionary this takes an extremely long time. 

freqDict = {}
for pair, count in pairs.iteritems(): #filter dictionary for only items with support greater than level set
    if count > support: 
        freqDict[pair] = count

print "Top 50 item pairs:"
print '\n'
print 'Item Pair' + '\t' + 'Support Count' + '\t' + 'Relative Support Count'
print '\n'
        
sortedFreqDict = sorted(freqDict.items(), key = lambda x: (-x[1], x[0])) #sort results by number and by lexicographic order
for i in range(50):
    print sortedFreqDict[i][0] + '\t' + str(sortedFreqDict[i][1]) + '\t' + str(float(sortedFreqDict[i][1])/totalBaskets)

Overwriting reducer-3-4.py


In [26]:
time !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper-3-4.py \
-reducer reducer-3-4.py \
-input ProductPurchaseData.txt \
-output sortedProducts

16/02/01 17:56:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:56:36 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:56:36 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:56:36 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:56:37 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 17:56:37 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 17:56:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local350127535_0001
16/02/01 17:56:37 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 17:56:37 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.FileOutputCommitter
16/02/01 17:56:37 INFO mapreduce.Job: The url to track the job: http:/

In [27]:
#show results
!hdfs dfs -cat /user/dunmireg/sortedProducts/part-00000

16/02/01 17:57:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Top 50 item pairs:	
	
	
Item Pair	Support Count	Relative Support Count
	
	
DAI62779 ELE17451	1592	0.0511880646925
FRO40251 SNA80324	1412	0.0454004694383
DAI75645 FRO40251	1254	0.0403202469374
FRO40251 GRO85051	1213	0.0390019613517
DAI62779 GRO73461	1139	0.0366226166361
DAI75645 SNA80324	1130	0.0363332368734
DAI62779 FRO40251	1070	0.0344040384554
DAI62779 SNA80324	923	0.0296775023311
DAI62779 DAI85309	918	0.0295167357963
ELE32164 GRO59710	911	0.0292916626475
DAI62779 DAI75645	882	0.0283592167454
FRO40251 GRO73461	882	0.0283592167454
DAI62779 ELE92920	877	0.0281984502106
FRO40251 FRO92469	835	0.026848011318
DAI62779 ELE32164	832	0.0267515513971
DAI75645 GRO73461	712	0.0228931545609
DAI43223 ELE32164	711	0.022861001254
DAI62779 GRO30386	709	0.02279669464
ELE17451 FRO40251	697	0.0224108549564
DAI85309 ELE99737	659	0.0211890292917
DAI62779 ELE26

The total time taken is displayed here: 

CPU times: user 112 ms, sys: 28.3 ms, total: 140 ms
Wall time: 21.4 s

This code used 1 mapper and 1 reducer. This was run on an 8 GB Macbook Air with a 2.2 GHz Intel Core i7 quad core processor.

-----

## HW3.5
*Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.*

*Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)*

*Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs*

### Solution Approach A: 

### Mapper

The mapper for the stripes implementation is noticeably different than the one used for the pairs implementation. In particular, a dictionary stripe is emitted which is then parsed literally by the reducer or optional combiner. From here, the stripes are aggregated per token then divided by two to compensate for the two combinations in which a key can be updated. 

In [1]:
%%writefile mapperQ35.py
#!/usr/bin/env python

import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	# Read input 
	data = readInput(sys.stdin)
	totalBaskets = 0
    
	for line in data: 
		
		totalBaskets += 1
		
		for token in line: 

			occurrence = defaultdict(int) 

			# Remove token from neighbors
			stripe = [x for x in line if x != token]
			
			# Create co-occurrence array 
			for neighbor in stripe: 
				occurrence[neighbor] += 1

			# Emit
			cArray = dict(occurrence)

			print '%s%s%s' % (token, '\t', str(cArray))

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Mappers,1\n")

	# Emit basket count
	print '%s%s%s' % ('*', '\t', str(totalBaskets))

Overwriting mapperQ35.py


### Reducer

Here, the reducer has been changed to compensate for the adjusted input. The reducer aggregates the stripes by aggregating the incoming dictionaries. It does this efficiently using `defaultdict`, an automatically instantiating key-value dictionary included in `collections`. 

In [8]:
%%writefile reducerQ35.py 
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
from itertools import combinations, chain 
import ast 


# Read input from mapper 
def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)	


if __name__ == "__main__":

	tokenDict = defaultdict(list)
	mergedDict = defaultdict(dict)
	tupleList = defaultdict(float)
	flattenedTerms = defaultdict(int)
	support = 100
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		
		token = line[0]
		if token == '*': 

			# Increment basket 
			totalBaskets += int(line[1])	

		else:

			# Parse normally 
			stripe = ast.literal_eval(line[1])
				
			# Combine dictionaries 
			tokenDict[token].append(stripe) 


	# Merge stripes 
	for k, v in tokenDict.iteritems(): 
		merged = defaultdict(int)
		
		# Loop and aggregate 
		for stripe in v: 
			for stripeKey in stripe: 
				merged[stripeKey] += stripe[stripeKey]

		mergedDict[k] = merged


	# Create key-value pairs 
	for token, stripe in mergedDict.iteritems(): 
		for innerToken, count in stripe.iteritems(): 

			# Get unique keys 
			tokenPair = [token, innerToken]
			tokenPair.sort()
			tuplePair = tuple(tokenPair)

			# Overcounting exactly twice per pair
			tupleList[tuplePair] += count / 2
	


	# Find most frequent terms and filter
	mostFrequentTerms = [(k, int(v), round(int(v) / totalBaskets, 3)) for k, v in tupleList.iteritems() 
                         if int(v) >= support]
    
	mostFrequentTerms = sorted(mostFrequentTerms, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Terms ==========' + '\n'
	template = "{0:30}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "RELATIVE FREQUENCY")
	
	# Print terms 
	for termPair in mostFrequentTerms[:50]: 
		print template.format(*termPair)

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Reducers,1\n")        

Overwriting reducerQ35.py


### Combiner

Below is the combiner used optionally in the process. It aggregates in a similar way to the reduce and outputes partially aggregated stripes. 

In [3]:
%%writefile combinerQ35.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict 
import ast 


# Read input from mapper 
def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	tokenDict = defaultdict(list)
	mergedDict = defaultdict(dict)
	totalBaskets = 0


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		token = line[0]
		if token == '*': 

			# Increment basket 
			totalBaskets += int(line[1])	

		else:

			# Parse normally 
			stripe = ast.literal_eval(line[1])
				
			# Combine dictionaries 
			tokenDict[token].append(stripe) 


	# # Merge stripes 
	for k, v in tokenDict.iteritems(): 
		merged = defaultdict(int)
		
		# Loop and aggregate 
		for stripe in v: 
			for stripeKey in stripe: 
				merged[stripeKey] += stripe[stripeKey]

		mergedDict[k] = dict(merged)

	# Emit results 
	for k, v in mergedDict.iteritems(): 
		print '%s%s%s' % (k, '\t', str(v))

	# Update counter 
	sys.stderr.write("reporter:counter:User-Defined,Number of Combiners,1\n")      

	# Emit baskets 
	print '%s%s%s' % ('*', '\t', str(totalBaskets))          

Overwriting combinerQ35.py


##### Submit Job 

Our wrapper remains unchanged from the prior implementation that has an argument input for the combiner. We re-use it here:

In [9]:
!bash wrapperQ34.sh ProductPurchaseData.txt mapperQ35.py reducerQ35.py combinerQ35.py

16/02/01 19:50:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:50:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/02/01 19:50:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:50:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:50:42 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/02/01 19:50:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ35.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

The job was run on a Macbook Pro with a 2.66GHz Intel dual-core processor and 4GB of memory. 

This job took 1 minute and 51 seconds, so only slighly longer than the previous job. 

Based on the results from Question 3.4 and Question 3.5, the two jobs use the same number of combiners, mappers, and reducers to accomplish their respective tasks. These are 2 combiners, 1 mapper, and 1 reducer for both jobs. 

-----

## HW3.6 (Optional)

### Part A

*What is the Apriori algorithm? Describe an example use in your domain of expertise.*

#### Solution: 

The Apriori algorithm is an algorithm for associative rule learning over transactional data sets. In particular, it implements a multi-scan approach with different tuning parameters like minimum support and minimum confidence. These are used to improve efficiency and control performance. In particular, the Apriori algorithm will identify frequent terms from the given support, and recurisvely compute larger baskets and search for additional frequent items. The Apriori algorithm stops when no further baskets can be found that are frequent. 

Within the domain of telecommunications, Apriori-like algorithms are used to describe the customer journey from signing-up to porting out. Identifying patterns that lead to churn is very important for Big Telecom, as the market continues to grow more volatile and competitive. Looking at information like clicks, calls to customer support, or billing events and building associative rules is important in finding business insights that improve customer retention. 


### Part B
*Define confidence and lift*

#### Solution: 

Given elements A and B, the confidence of A => B is equal to the support of A and B divided by the support of A. In words, confidence represents the likelihood of observing a basket containing B given a basket containing A. 

Lift is defined as a measure of performance of a model at predicting enhanced responses against a random choice targetting. In other words, Lift describes how well a model performs in identifying association rules that are more confident than selecting association rules at random. Given elements A and B, the lift of A => B is defined as the confidence of A and B given the average confidence across the entire basket list. 

-----

## HW3.7. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a program using the A-priori algorithm
to find products which are frequently browsed together. Fix the support to s = 100 
(i.e. product sets need to occur together at least 100 times to be considered frequent) 
and find itemsets of size 2 and 3.

Then extract association rules from these frequent items. 

A rule is of the form: 

(item1, item5) ⇒ item2.

List the top 10 discovered rules in descreasing order of confidence in the following format
 
(item1, item5) ⇒ item2, supportCount ,support, confidence

### HW3.7 Mapper

We use the pairs implementation here as it requires minimal code changes, albeit being less efficient. To this end, the mapper now outputs both 2 and 3-tuples which are treated the same by the reducer. The reducer changes its printing functionality to this end.

In [91]:
%%writefile mapperQ37.py
#!/usr/bin/env python

from __future__ import division 
import sys
from collections import defaultdict
from itertools import combinations

def readInput(file, separator=None):
	for line in file:
		yield line.split(separator)

if __name__ == "__main__":

	# Read input 
	data = readInput(sys.stdin)
	for line in data: 

		# Get unique keys 
		doublePairs = list(combinations(line, 2))
		triplePairs = list(combinations(line, 3))
		

		# Sort doubles 
		sortedDoubles = []
		for pair in doublePairs: 
			pList = list(pair)
			pList.sort()
			sortedDoubles.append(tuple(pList))

		# Emit
		for pair in sortedDoubles: 
			print '%s%s%s' % (pair, '\t', 1)


		# Sort triples 
		sortedTriples = []
		for pair in triplePairs: 
			pList = list(pair)
			pList.sort()
			sortedTriples.append(tuple(pList))

		# Emit
		for pair in sortedTriples: 
			print '%s%s%s' % (pair, '\t', 1)

Overwriting mapperQ37.py


### HW3.7 Reducer

We modify the reducer so it outputs the top 50 of both the length-2 and length-3 tuples. This is mainly a cosmetic change in the printing functionality. 

In [97]:
%%writefile reducerQ37.py
#!/usr/bin/env python

from __future__ import division 
import sys 
from collections import defaultdict
import ast 


def readInput(file, separator='\t'):
	for line in file:
		yield line.split(separator)


if __name__ == "__main__":

	# Final store 
	storingDict = defaultdict(int)
	support = 100


	# Read data 
	data = readInput(sys.stdin)
	for line in data: 

		# Parse value 
		token = line[0]
		termCount = int(line[1])


		# Store results 
		storingDict[token] += termCount

	
	# Filter
	filterDict = defaultdict(int)
	for k, v in storingDict.iteritems(): 
		if v >= support:
			filterDict[k] += v 


	# Find most frequent doubles 
	mfqDoubles = [(k, v, support) for k, v in filterDict.iteritems()
					if k.count(',') == 1]

	mfqDoubles = sorted(mfqDoubles, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Doubles ==========' + '\n'
	template = "{0:30}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "SUPPORT")
	
	# Print terms 
	for termPair in mfqDoubles[:50]: 
		print template.format(*termPair)



	# Find most frequent triples 
	mfqTriples = [(k, v, support) for k, v in filterDict.iteritems()
					if k.count(',') == 2]
					
	mfqTriples = sorted(mfqTriples, 
								key = lambda x: x[1], 
								reverse = True)


	# Get results 
	print '\n' + '========== Most Frequent Triples ==========' + '\n'
	template = "{0:50}|{1:20}|{2:20}"
	print template.format("PAIR", "SUPPORT COUNT", "SUPPORT")
	
	# Print terms 
	for termPair in mfqTriples[:50]: 
		print template.format(*termPair)


Overwriting reducerQ37.py


### HW3.7 Hadoop MapReduce Submit Job 

We now submit the job using the combiner and wrapper defined in Question 3.4. 

In [98]:
!bash wrapperQ34.sh ProductPurchaseData.txt mapperQ37.py reducerQ37.py combinerQ34.py

16/01/30 20:39:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 20:39:33 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/john/notebook
16/01/30 20:39:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 20:39:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 20:39:42 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/30 20:39:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/Users/john/Dropbox/MIDS/W261/Week3/Homework3/notebook/mapperQ37.py, /Users/john/Dropbox/MIDS/W261/Week3/Homework

-----

## Question 3.8 (Optional)
**

#### Solution: 

Here's some text.